# 疫学的因果関係
統計学が物的証拠として使われる代表例としては法律の決定や他何らかのトラブルに対する因果関係を証明するために疫学的因果関係を推定する。ここで、疫学的因果関係は7つの構成要素がある。本項では7つの構成要素の中で「一致性」「整合性」「必要条件」「十分条件」以外についての文面をそのままプログラムとして実装する。

また、疫学における因果関係はその専門分野の知識を多く持っていなくても使用できるメリットがあるため、研究活動において最初の頃には役に立ちやすい。

なお、適切なデータセットが見つからなかったため項目によって使用するデータセットを変えている。

参考1：https://chuo-kentetsu.co.jp/cgk/topix/tecrep30.pdf

参考2：https://apps.who.int/iris/bitstream/handle/10665/43541/9241547073_jpn.pdf


## ライブラリのインポート

In [1]:
from sklearn.linear_model import LogisticRegression as LR
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 順序関係
原因と結果についての順序関係を正しく把握する。例えばX→Yが正しいのかY→Xが正しいかなど、ある事象に対する因果関係における順序について考察する。

ここでは一例としてゴルフの来場について、天気が原因でゴルフを結果として考えてどの天気の時にゴルフをするか、また結果と原因の順序が変わることで数値の変動を確認する。

### データの確認

In [2]:
df = pd.read_csv("golf.csv", encoding="shift-jis")
df.head()

,天気,気温,湿度,風,ゴルフ
0,晴,29,85,弱,しない
1,晴,27,90,強,しない
2,曇,28,78,弱,する
3,雨,21,96,弱,する
4,雨,20,80,弱,する


### ベイズの定理で原因と結果の確率を計算する

In [3]:
x_name = "天気"
y_name = "ゴルフ"

In [4]:
x_val = list(set(df[x_name].values.tolist()))
for x in x_val:
    dfx = df.query("%s=='%s'"%(x_name, x))
    y_val = list(set(dfx[y_name].values.tolist()))
    for y in y_val:
        dfxy = dfx.query("%s=='%s'"%(y_name, y))
        print("p(%s=%s | %s=%s) = %.2f"%(y_name, y, x_name, x, len(dfxy)/len(dfx)))

p(ゴルフ=する | 天気=雨) = 0.60
p(ゴルフ=しない | 天気=雨) = 0.40
p(ゴルフ=する | 天気=晴) = 0.40
p(ゴルフ=しない | 天気=晴) = 0.60
p(ゴルフ=する | 天気=曇) = 1.00


In [5]:
y_val = list(set(df[y_name].values.tolist()))
for y in y_val:
    dfy = df.query("%s=='%s'"%(y_name, y))
    x_val = list(set(dfy[x_name].values.tolist()))
    for x in x_val:
        dfyx = dfy.query("%s=='%s'"%(x_name, x))
        print("p(%s=%s | %s=%s) = %.2f"%(x_name, x, y_name, y, len(dfyx)/len(dfy)))

p(天気=曇 | ゴルフ=する) = 0.44
p(天気=晴 | ゴルフ=する) = 0.22
p(天気=雨 | ゴルフ=する) = 0.33
p(天気=晴 | ゴルフ=しない) = 0.60
p(天気=雨 | ゴルフ=しない) = 0.40


### 結果の解釈
この結果から曇が原因でゴルフを行う事が結果になることが分かる。元々100%だった「曇→ゴルフ」が「ゴルフ→曇」にした場合は44%に変動し、曇が順序として原因と分かる。

## 量反応関係
調べたい量的変数を選び相関係数を求める。ただし、条件によって変わる可能性を考慮して質的変数で条件を絞り、それによって相関に違いが現れるかを確認する。また、ロジスティック回帰で係数からオッズ比を計算する事で各項目の量的な判別基準を算出して特徴を把握する。

ここでは一例としてアヤメの花の可弁の長さとがくの長さに関する相関について品種ごとに分けて計測する。

### データの確認

In [6]:
df = pd.read_csv("iris.csv")
df.head()

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2


### オッズ比

In [7]:
y_name = "category"
y = df[y_name].values
x_table = df.drop(y_name, axis=1)
x_name = x_table.columns
x = x_table.values
model = LR()
model.fit(x, y)
coef = model.coef_
coef = np.vstack((coef, np.exp(coef)))
col = []
for v in (list(set(y))):
    col.append("%dの回帰係数"%(v))
for v in (list(set(y))):
    col.append("%dのオッズ比"%(v))
df_coef = pd.DataFrame(coef.T)
df_coef.columns = col
df_coef.index = x_name
df_coef

,0の回帰係数,1の回帰係数,2の回帰係数,0のオッズ比,1のオッズ比,2のオッズ比
sepal length (cm),-0.423395,0.534093,-0.110698,0.654820,1.705901,0.895209
sepal width (cm),0.961689,-0.317924,-0.643765,2.616112,0.727658,0.525311
petal length (cm),-2.519421,-0.205372,2.724794,0.080506,0.814344,15.253267
petal width (cm),-1.086077,-0.939581,2.025658,0.337538,0.390792,7.581094


### 結果の解釈
基本的にがくの長さと花弁の長さは相関関係にある事が分かる。しかし品種「0」の時に相関が弱いため、品種「0」の場合に相関が働かない何らかの要因があると考えられる。

また、オッズ比から数値の反応を考慮すると、品種「0」ではどちらも長さは1より小さく一方で「1」と品種「2」はどちらかが1を上回ることが分かる。

## 強固性
リスク比が大きく原因と結果の強い関連性が高い場合は当該要因が原因である可能性がある。そこで、強固性の基準としては相対危険度を用いる。

ここでは一例として米国メリーランド州の刑務所から釈放され釈放後に一年間追跡調査された受刑者に関するデータから労働経験の有無(1：有、0：無)と逮捕の有無(1：有、0：無)で相対危険度を算出する。

### データの確認

In [8]:
df = pd.read_csv("rossi.csv")
df.head()

,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


### 相対危険度の算出

In [9]:
cross = pd.crosstab(df["wexp"], df["arrest"])
cross

arrest,0,1
wexp,,
0,123,62
1,195,52


In [10]:
aw = cross.values
print("相対危険度：%.2f"%((aw[0][1]/sum(aw[0]))/((aw[1][1]/sum(aw[1])))))

相対危険度：1.59


### 結果の解釈
相対危険度から労働経験がない人は労働経験がある人に比べて1.59倍逮捕されていることが確認できる。そのため、労働しない事によって何らかの犯罪の原因が発生する事が考えられる。